In [5]:
import pandas as pd

In [6]:
import hopsworks

# Initialize connection to Hopsworks using API key
project = hopsworks.login(api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0")
# Access the feature store for the project
fs = project.get_feature_store()

# Retrieve the feature group
final_merged_fg = fs.get_feature_group(name="final_merged", version=1)

# Read the feature group as a DataFrame
final_merged_df = final_merged_fg.read()



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1104021
2024-10-19 19:22:46,067 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-10-19 19:22:46,067 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.81s) 


In [7]:
final_merged_df.head(5)

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,50064164,R-20376100,2019-02-12 07:00:00,2019-02-12 22:25:12,0,74.000000,4.500000,0.000000,81.0,6.000000,...,Sandra Wilson,female,44.0,13.0,conservative,6.0,50064164.0,44.74,0,8518
1,60858923,R-461413e3,2019-01-01 07:00:00,2019-01-07 00:55:48,0,68.785714,11.142857,0.000000,71.0,5.500000,...,Jose Martin,male,52.0,21.0,proactive,4.0,60858923.0,60.74,1,7091
2,24859831,R-18468971,2019-01-13 07:00:00,2019-01-13 23:40:12,0,53.000000,11.500000,0.000000,75.5,6.000000,...,Luke Kennedy,male,42.0,2.0,proactive,7.0,24859831.0,60.48,0,5129
3,31827682,R-6f2f34a8,2019-01-16 07:00:00,2019-01-16 13:04:48,0,78.000000,9.000000,0.033333,79.0,5.333333,...,George Webb,male,45.0,16.0,proactive,9.0,31827682.0,59.97,0,6443
4,30822651,R-c5c0de8d,2019-01-25 07:00:00,2019-01-26 05:13:12,1,65.200000,8.000000,0.040000,90.6,5.200000,...,Kimberly Torres,female,55.0,24.0,conservative,9.0,30822651.0,54.19,1,6231


In [8]:
final_merged_df.shape

(12265, 49)

In [9]:
final_merged_df.info()  # Verify the structure



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12265 entries, 0 to 12264
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   truck_id                        12265 non-null  int64  
 1   route_id                        12265 non-null  object 
 2   departure_date                  12265 non-null  object 
 3   estimated_arrival               12265 non-null  object 
 4   delay                           12265 non-null  int64  
 5   route_avg_temp                  12265 non-null  float64
 6   route_avg_wind_speed            12265 non-null  float64
 7   route_avg_precip                12265 non-null  float64
 8   route_avg_humidity              12265 non-null  float64
 9   route_avg_visibility            12265 non-null  float64
 10  route_avg_pressure              12265 non-null  float64
 11  route_description               12265 non-null  object 
 12  estimated_arrival_nearest_hour  

In [10]:
import pandas as pd

# Assuming df is your DataFrame
# Step 1: Convert object types to datetime
datetime_columns = [
    'departure_date', 
    'estimated_arrival', 
    'estimated_arrival_nearest_hour', 
    'departure_date_nearest_hour'
]

for col in datetime_columns:
    final_merged_df[col] = pd.to_datetime(final_merged_df[col], errors='coerce')  # Coerce errors to handle any invalid dates

# Step 2: Adjust data types for identifiers and descriptions
final_merged_df['origin_id'] = final_merged_df['origin_id'].astype('object')
final_merged_df['destination_id'] = final_merged_df['destination_id'].astype('object')
final_merged_df['origin_description'] = final_merged_df['origin_description'].astype('object')
final_merged_df['destination_description'] = final_merged_df['destination_description'].astype('object')

# Step 3: Validation
# Check for expected columns
expected_columns = [
    'unique_id', 'truck_id', 'route_id', 'departure_date', 'estimated_arrival',
    'delay', 'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
    'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
    'route_description', 'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
    'origin_id', 'destination_id', 'distance', 'average_hours', 'origin_temp',
    'origin_wind_speed', 'origin_description', 'origin_precip', 'origin_humidity',
    'origin_visibility', 'origin_pressure', 'destination_temp', 'destination_wind_speed',
    'destination_description', 'destination_precip', 'destination_humidity',
    'destination_visibility', 'destination_pressure', 'avg_no_of_vehicles',
    'accident', 'truck_age', 'load_capacity_pounds', 'mileage_mpg', 'fuel_type',
    'driver_id', 'name', 'gender', 'age', 'experience', 'driving_style', 
    'ratings', 'vehicle_no', 'average_speed_mph', 'is_midnight'
]

# Validate columns
missing_columns = [col for col in expected_columns if col not in final_merged_df.columns]
if missing_columns:
    print(f"Missing columns in DataFrame: {missing_columns}")
else:
    print("All expected columns are present.")

# Check for non-null values in critical columns
non_null_counts = final_merged_df[datetime_columns].isnull().sum()
print("Non-null counts for datetime columns after conversion:")
print(non_null_counts)

# Final output
print("Data types after adjustments:")
print(final_merged_df.dtypes)


All expected columns are present.
Non-null counts for datetime columns after conversion:
departure_date                        0
estimated_arrival                     0
estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
dtype: int64
Data types after adjustments:
truck_id                                   int64
route_id                                  object
departure_date                    datetime64[ns]
estimated_arrival                 datetime64[ns]
delay                                      int64
route_avg_temp                           float64
route_avg_wind_speed                     float64
route_avg_precip                         float64
route_avg_humidity                       float64
route_avg_visibility                     float64
route_avg_pressure                       float64
route_description                         object
estimated_arrival_nearest_hour    datetime64[ns]
departure_date_nearest_hour       datetime64[ns]
origin_id             

In [11]:
final_merged_df.info()  # Verify the structure



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12265 entries, 0 to 12264
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   truck_id                        12265 non-null  int64         
 1   route_id                        12265 non-null  object        
 2   departure_date                  12265 non-null  datetime64[ns]
 3   estimated_arrival               12265 non-null  datetime64[ns]
 4   delay                           12265 non-null  int64         
 5   route_avg_temp                  12265 non-null  float64       
 6   route_avg_wind_speed            12265 non-null  float64       
 7   route_avg_precip                12265 non-null  float64       
 8   route_avg_humidity              12265 non-null  float64       
 9   route_avg_visibility            12265 non-null  float64       
 10  route_avg_pressure              12265 non-null  float64       
 11  ro

In [12]:
# Check data types and null values
data_types = final_merged_df.dtypes
null_counts = final_merged_df.isnull().sum()

# Summary of data types and null values
summary = pd.DataFrame({'Data Type': data_types, 'Null Values': null_counts})
print(summary)

# Check memory usage
memory_usage = final_merged_df.memory_usage(deep=True)
print(f"Memory Usage:\n{memory_usage}")


                                     Data Type  Null Values
truck_id                                 int64            0
route_id                                object            0
departure_date                  datetime64[ns]            0
estimated_arrival               datetime64[ns]            0
delay                                    int64            0
route_avg_temp                         float64            0
route_avg_wind_speed                   float64            0
route_avg_precip                       float64            0
route_avg_humidity                     float64            0
route_avg_visibility                   float64            0
route_avg_pressure                     float64            0
route_description                       object            0
estimated_arrival_nearest_hour  datetime64[ns]        12265
departure_date_nearest_hour     datetime64[ns]        12265
origin_id                               object            0
destination_id                          

In [13]:
final_merged_df.isnull().sum()  # Check for null values

truck_id                              0
route_id                              0
departure_date                        0
estimated_arrival                     0
delay                                 0
route_avg_temp                        0
route_avg_wind_speed                  0
route_avg_precip                      0
route_avg_humidity                    0
route_avg_visibility                  0
route_avg_pressure                    0
route_description                     0
estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
origin_id                             0
destination_id                        0
distance                           1600
average_hours                      1600
origin_temp                        1600
origin_wind_speed                  1600
origin_description                12265
origin_precip                      1600
origin_humidity                    1600
origin_visibility                  1600
origin_pressure                    1600


In [14]:
# Check for null values
null_counts = final_merged_df.isnull().sum()
print(null_counts[null_counts > 0])
final_merged_df.dtypes

estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
distance                           1600
average_hours                      1600
origin_temp                        1600
origin_wind_speed                  1600
origin_description                12265
origin_precip                      1600
origin_humidity                    1600
origin_visibility                  1600
origin_pressure                    1600
destination_temp                   1600
destination_wind_speed             1600
destination_description           12265
destination_precip                 1600
destination_humidity               1600
destination_visibility             1600
destination_pressure               1600
avg_no_of_vehicles                 1607
accident                           1600
load_capacity_pounds                659
fuel_type                           659
name                                848
gender                              848
age                                 848


truck_id                                   int64
route_id                                  object
departure_date                    datetime64[ns]
estimated_arrival                 datetime64[ns]
delay                                      int64
route_avg_temp                           float64
route_avg_wind_speed                     float64
route_avg_precip                         float64
route_avg_humidity                       float64
route_avg_visibility                     float64
route_avg_pressure                       float64
route_description                         object
estimated_arrival_nearest_hour    datetime64[ns]
departure_date_nearest_hour       datetime64[ns]
origin_id                                 object
destination_id                            object
distance                                 float64
average_hours                            float64
origin_temp                              float64
origin_wind_speed                        float64
origin_description  

In [15]:
final_merged_df.shape

(12265, 49)

In [16]:
from sklearn.impute import SimpleImputer


#For numerical columns, you might fill with the mean or median
numerical_cols = final_merged_df.select_dtypes(include=['float64', 'int64']).columns

# Fill with mean
final_merged_df[numerical_cols] = final_merged_df[numerical_cols].fillna(final_merged_df[numerical_cols].mean())


# # For numerical features, use mean imputation
# num_imputer = SimpleImputer(strategy='mean')
# final_merged_df[['distance', 'average_hours', 'origin_temp', 'origin_wind_speed']] = num_imputer.fit_transform(
#     final_merged_df[['distance', 'average_hours', 'origin_temp', 'origin_wind_speed']]
# )

# For categorical features, use mode imputation
cat_imputer = SimpleImputer(strategy='most_frequent')
final_merged_df[['fuel_type', 'name', 'gender']] = cat_imputer.fit_transform(
    final_merged_df[['fuel_type', 'name', 'gender']]
)


In [17]:
# Impute missing values in the 'age' column with the median
final_merged_df['age'].fillna(final_merged_df['age'].median(), inplace=True)

# Verify that there are no more null values in the 'age' column
print(final_merged_df['age'].isnull().sum())


0


In [18]:
from sklearn.impute import SimpleImputer

# Imputation for numerical features
numerical_features = [
    'origin_precip', 'origin_humidity', 'origin_visibility', 'origin_pressure',
    'destination_temp', 'destination_wind_speed', 'destination_precip', 
    'destination_humidity', 'destination_visibility', 'destination_pressure', 
    'avg_no_of_vehicles', 'accident', 'load_capacity_pounds', 'average_speed_mph'
]
num_imputer = SimpleImputer(strategy='mean')  # or use 'median' if preferred
final_merged_df[numerical_features] = num_imputer.fit_transform(final_merged_df[numerical_features])

# Imputation for categorical features
categorical_features = [
    'fuel_type', 'name', 'gender', 'experience', 
    'driving_style', 'ratings', 'vehicle_no'
]
cat_imputer = SimpleImputer(strategy='most_frequent')
final_merged_df[categorical_features] = cat_imputer.fit_transform(final_merged_df[categorical_features])

# For the high missing columns, if you're keeping them, you might need a separate approach
# final_merged_df['estimated_arrival_nearest_hour'] = ... (use model predictions or other techniques)


In [19]:
final_merged_df.isnull().sum()  # Check for null values

truck_id                              0
route_id                              0
departure_date                        0
estimated_arrival                     0
delay                                 0
route_avg_temp                        0
route_avg_wind_speed                  0
route_avg_precip                      0
route_avg_humidity                    0
route_avg_visibility                  0
route_avg_pressure                    0
route_description                     0
estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
origin_id                             0
destination_id                        0
distance                              0
average_hours                         0
origin_temp                           0
origin_wind_speed                     0
origin_description                12265
origin_precip                         0
origin_humidity                       0
origin_visibility                     0
origin_pressure                       0


In [20]:
from sklearn.impute import KNNImputer

# Assuming 'categorical_columns' is a list of your categorical features
categorical_columns = ['fuel_type', 'name', 'gender', 'driving_style']
imputer = KNNImputer(n_neighbors=5)

# Only apply KNN imputer to numeric representations of categorical variables
# Transform categorical variables to numerical codes if needed
final_merged_df[categorical_columns] = final_merged_df[categorical_columns].apply(lambda col: col.astype('category').cat.codes)

# Fit and transform
final_merged_df[categorical_columns] = imputer.fit_transform(final_merged_df[categorical_columns])


In [21]:
null_counts_after = final_merged_df.isnull().sum()
print(null_counts_after[null_counts_after > 0])
null_counts_after.dtypes


estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
origin_description                12265
destination_description           12265
dtype: int64


dtype('int64')

In [22]:
# Imputation for categorical columns
final_merged_df['origin_description'].fillna(final_merged_df['origin_description'].mean(), inplace=True)
final_merged_df['destination_description'].fillna(final_merged_df['destination_description'].mean(), inplace=True)
final_merged_df['estimated_arrival_nearest_hour'].fillna(final_merged_df['estimated_arrival_nearest_hour'].mean(), inplace=True)
final_merged_df['departure_date_nearest_hour'].fillna(final_merged_df['departure_date_nearest_hour'].mean(), inplace=True)

In [23]:
final_merged_df.dtypes

truck_id                                   int64
route_id                                  object
departure_date                    datetime64[ns]
estimated_arrival                 datetime64[ns]
delay                                      int64
route_avg_temp                           float64
route_avg_wind_speed                     float64
route_avg_precip                         float64
route_avg_humidity                       float64
route_avg_visibility                     float64
route_avg_pressure                       float64
route_description                         object
estimated_arrival_nearest_hour    datetime64[ns]
departure_date_nearest_hour       datetime64[ns]
origin_id                                 object
destination_id                            object
distance                                 float64
average_hours                            float64
origin_temp                              float64
origin_wind_speed                        float64
origin_description  

In [24]:
final_merged_df.shape

(12265, 49)

In [25]:
# # Filling missing numerical values with mean
# numerical_cols = ['estimated_arrival_nearest_hour', 'departure_date_nearest_hour', 
#                   'origin_description', 'destination_description']

# final_merged_df[numerical_cols] = final_merged_df[numerical_cols].fillna(final_merged_df[numerical_cols].mean())

# # Filling missing categorical values with 'Unknown'
# categorical_cols = ['origin_description', 'destination_description', 'fuel_type']

# final_merged_df[categorical_cols] = final_merged_df[categorical_cols].fillna('Unknown')

# # Check the updated DataFrame for null values
# print(final_merged_df.isnull().sum())


In [26]:
# # Filling remaining numerical values with 0
# remaining_numerical_cols = ['estimated_arrival_nearest_hour', 'departure_date_nearest_hour', 
#                              'origin_id', 'destination_id', 'distance', 'average_hours', 
#                              'origin_temp', 'origin_wind_speed', 'origin_precip', 
#                              'origin_humidity', 'origin_visibility', 'origin_pressure', 
#                              'destination_temp', 'destination_wind_speed', 'destination_precip', 
#                              'destination_humidity', 'destination_visibility', 
#                              'destination_pressure', 'avg_no_of_vehicles', 'accident']

# final_merged_df[remaining_numerical_cols] = final_merged_df[remaining_numerical_cols].fillna(0)

# # Check the updated DataFrame for null values
# print(final_merged_df.isnull().sum())


In [27]:
final_merged_df.isnull().sum()  # Check for null values

truck_id                              0
route_id                              0
departure_date                        0
estimated_arrival                     0
delay                                 0
route_avg_temp                        0
route_avg_wind_speed                  0
route_avg_precip                      0
route_avg_humidity                    0
route_avg_visibility                  0
route_avg_pressure                    0
route_description                     0
estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
origin_id                             0
destination_id                        0
distance                              0
average_hours                         0
origin_temp                           0
origin_wind_speed                     0
origin_description                12265
origin_precip                         0
origin_humidity                       0
origin_visibility                     0
origin_pressure                       0


In [28]:
final_merged_df.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
       'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'destination_temp',
       'destination_wind_speed', 'destination_description',
       'destination_precip', 'destination_humidity', 'destination_visibility',
       'destination_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight', 'unique_id'],
      dt

In [29]:
# 4. Recheck for any remaining missing values (optional)
remaining_missing = final_merged_df.isnull().sum()
print("Remaining Missing Values:\n", remaining_missing[remaining_missing > 0])


Remaining Missing Values:
 estimated_arrival_nearest_hour    12265
departure_date_nearest_hour       12265
origin_description                12265
destination_description           12265
dtype: int64


In [30]:
final_merged_df.shape

(12265, 49)

In [31]:
# Check data types of all columns
print(final_merged_df.dtypes)


truck_id                                   int64
route_id                                  object
departure_date                    datetime64[ns]
estimated_arrival                 datetime64[ns]
delay                                      int64
route_avg_temp                           float64
route_avg_wind_speed                     float64
route_avg_precip                         float64
route_avg_humidity                       float64
route_avg_visibility                     float64
route_avg_pressure                       float64
route_description                         object
estimated_arrival_nearest_hour    datetime64[ns]
departure_date_nearest_hour       datetime64[ns]
origin_id                                 object
destination_id                            object
distance                                 float64
average_hours                            float64
origin_temp                              float64
origin_wind_speed                        float64
origin_description  

In [32]:
# Correcting the types of the categorical columns
categorical_cols = [
    'origin_description',
    'destination_description',
    'fuel_type',
    'name',
    'gender',
    'experience',
    'driving_style',
    'ratings',
    'vehicle_no'
]

for col in categorical_cols:
    final_merged_df[col] = final_merged_df[col].astype('object')

# Optional: Verify the data types again
print(final_merged_df.dtypes)


truck_id                                   int64
route_id                                  object
departure_date                    datetime64[ns]
estimated_arrival                 datetime64[ns]
delay                                      int64
route_avg_temp                           float64
route_avg_wind_speed                     float64
route_avg_precip                         float64
route_avg_humidity                       float64
route_avg_visibility                     float64
route_avg_pressure                       float64
route_description                         object
estimated_arrival_nearest_hour    datetime64[ns]
departure_date_nearest_hour       datetime64[ns]
origin_id                                 object
destination_id                            object
distance                                 float64
average_hours                            float64
origin_temp                              float64
origin_wind_speed                        float64
origin_description  

Test_validation_split

In [33]:
import pandas as pd
import datetime

# Get the current UTC time
current_time = datetime.datetime.now(datetime.timezone.utc)

# Convert estimated_arrival and departure_date to timezone-aware datetime
final_merged_df['estimated_arrival'] = pd.to_datetime(final_merged_df['estimated_arrival'], utc=True)
final_merged_df['departure_date'] = pd.to_datetime(final_merged_df['departure_date'], utc=True)

# Define the training, validation, and test datasets based on today's date
train_df = final_merged_df[final_merged_df['estimated_arrival'] <= current_time]
validation_df = final_merged_df[(final_merged_df['estimated_arrival'] > current_time) & 
                                 (final_merged_df['estimated_arrival'] <= current_time + pd.Timedelta(days=7))]
test_df = final_merged_df[final_merged_df['estimated_arrival'] > current_time + pd.Timedelta(days=7)]


In [34]:
import pandas as pd
import datetime

# Get the current UTC time
current_time = datetime.datetime.now(datetime.timezone.utc)

# Convert estimated_arrival and departure_date to timezone-aware datetime
final_merged_df['estimated_arrival'] = pd.to_datetime(final_merged_df['estimated_arrival'], utc=True)
final_merged_df['departure_date'] = pd.to_datetime(final_merged_df['departure_date'], utc=True)

# Define the training, validation, and test datasets based on the current date
train_df = final_merged_df[final_merged_df['estimated_arrival'] <= current_time]
validation_df = final_merged_df[(final_merged_df['estimated_arrival'] > current_time) & 
                                 (final_merged_df['estimated_arrival'] <= current_time + pd.Timedelta(days=7))]
test_df = final_merged_df[final_merged_df['estimated_arrival'] > current_time + pd.Timedelta(days=7)]



In [35]:
print("Minimum estimated_arrival:", final_merged_df['estimated_arrival'].min())
print("Maximum estimated_arrival:", final_merged_df['estimated_arrival'].max())
print("Current UTC time:", current_time)
print("Total rows in final_merged_df:", final_merged_df.shape[0])


Minimum estimated_arrival: 2019-01-01 07:04:48+00:00
Maximum estimated_arrival: 2019-02-14 16:06:00+00:00
Current UTC time: 2024-10-19 23:23:13.838043+00:00
Total rows in final_merged_df: 12265


In [36]:
import pandas as pd
import numpy as np

# Assuming final_merged_df is already defined

# Step 1: Define total rows
total_rows = len(final_merged_df)

# Step 2: Calculate the number of training, validation, and testing samples
train_size = int(total_rows * 0.7)    # 70%
validation_size = int(total_rows * 0.1) # 10%
test_size = total_rows - (train_size + validation_size)  # 20%

# Step 3: Shuffle the data
shuffled_df = final_merged_df.sample(frac=1, random_state=42).reset_index(drop=True)



In [37]:
# Step 4: Split the data into training, validation, and testing sets
train_df = shuffled_df.iloc[:train_size]
validation_df = shuffled_df.iloc[train_size:train_size + validation_size]
test_df = shuffled_df.iloc[train_size + validation_size:]



In [38]:
# Step 5: Define features and target
cts_cols = ['route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip', 
            'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure', 
            'distance', 'average_hours', 'origin_temp', 'origin_wind_speed', 
            'origin_precip', 'origin_humidity', 'origin_visibility', 
            'origin_pressure', 'destination_temp', 'destination_wind_speed', 
            'destination_precip', 'destination_humidity', 'destination_visibility', 
            'destination_pressure', 'avg_no_of_vehicles', 'truck_age', 
            'load_capacity_pounds', 'mileage_mpg', 'age', 'experience', 
            'average_speed_mph']

cat_cols = ['route_description', 'fuel_type', 'gender', 
            'driving_style', 'ratings', 'is_midnight']

target = 'delay'




In [39]:
# Step 6: Splitting features and target variable
X_train = train_df[cts_cols + cat_cols]
y_train = train_df[target]

X_valid = validation_df[cts_cols + cat_cols]
y_valid = validation_df[target]

X_test = test_df[cts_cols + cat_cols]
y_test = test_df[target]



In [40]:
# Step 7: Output the shapes to confirm
print("Train set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_valid.shape, y_valid.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Train set shape: (8585, 33) (8585,)
Validation set shape: (1226, 33) (1226,)
Test set shape: (2454, 33) (2454,)


encoding and scaling

In [41]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define categorical and numerical columns
encoder_columns = cat_cols  # Categorical features
scaler_columns = cts_cols    # Continuous features

# Step 1: One-hot encoding for categorical features
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')



In [42]:
# Step 2: Standard scaling for continuous features
scaler = StandardScaler()

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, scaler_columns),
        ('cat', encoder, encoder_columns)
    ],
    remainder='drop'  # Drop any other columns not specified
)




In [43]:
# Step 3: Fit and transform the training data
X_train_transformed = preprocessor.fit_transform(X_train)

# Transform validation and test data using the same preprocessor
X_valid_transformed = preprocessor.transform(X_valid)
X_test_transformed = preprocessor.transform(X_test)

# Now X_train_transformed, X_valid_transformed, and X_test_transformed are ready for modeling

In [44]:
X_train_transformed.shape

(8585, 86)

In [45]:
print(final_merged_df.columns)


Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
       'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'destination_temp',
       'destination_wind_speed', 'destination_description',
       'destination_precip', 'destination_humidity', 'destination_visibility',
       'destination_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight', 'unique_id'],
      dt

 Set Up MLflow for Model Experimentation

build model with mlflow

In [42]:
pip install mlflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


import necessary libraries

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import mlflow.sklearn


2. Define Your Training and Validation Data

In [47]:
# Assuming you already have X_train_transformed, y_train, X_valid_transformed, y_valid


3. Train and Evaluate Each Model


A. Logistic Regression

logistic regression- with mlflow and hopswork model registry

In [50]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Step 2: Start MLflow run and train the Logistic Regression model
with mlflow.start_run() as run:
    # Step 3: Initialize and train the Logistic Regression model
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train_transformed, y_train)

    # Make predictions on validation set
    y_pred_log_reg = log_reg.predict(X_valid_transformed)

    # Step 4: Evaluate the model
    print("Logistic Regression Classification Report:")
    print(classification_report(y_valid, y_pred_log_reg))

    # Calculate accuracy scores
    train_score = accuracy_score(y_train, log_reg.predict(X_train_transformed))
    test_score = accuracy_score(y_valid, y_pred_log_reg)

    print(f"Train Score: {train_score}")
    print(f"Validation/Test Score: {test_score}")

    # Step 5: Log model and parameters in MLflow
    mlflow.sklearn.log_model(log_reg, "Logistic_Regression_Model")
    mlflow.log_params({"model": "Logistic Regression", "max_iter": 1000})
    mlflow.log_metrics({"train_score": train_score, "test_score": test_score})

    # Get MLflow run details
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 6: Save the model locally
    model_path = '/tmp/logistic_regression_model.pkl'
    joblib.dump(log_reg, model_path)

    # Step 7: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="Logistic_Regression_Model",
        version=1,  # Version of the model
        metrics={"train_score": train_score, "validation_score": test_score},
        description="Logistic Regression Model for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("Model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.82      0.82       813
           1       0.64      0.62      0.63       413

    accuracy                           0.76      1226
   macro avg       0.73      0.72      0.72      1226
weighted avg       0.75      0.76      0.76      1226

Train Score: 0.7380314502038439
Validation/Test Score: 0.7561174551386624
2024-10-19 19:24:11,423 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



2024/10/19 19:24:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run ID: b499ecfc56c348e396b698f22c8bc74d
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/b499ecfc56c348e396b698f22c8bc74d


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1551 elapsed<00:00 remaining<?

2024/10/19 19:24:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-mare-39 at: http://127.0.0.1:5000/#/experiments/0/runs/b499ecfc56c348e396b698f22c8bc74d.
2024/10/19 19:24:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/Logistic_Regression_Model/1
Model saved and registered in Hopsworks successfully!


hyperparameter tuning - logistic regression

In [55]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Ensure previous run is ended
mlflow.end_run()  

# Step 2: Start MLflow run
with mlflow.start_run() as run:
    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],  
        'penalty': ['l1', 'l2'],       
        'solver': ['liblinear', 'saga']  
    }

    # Initialize GridSearchCV
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

    # Fit on training data
    grid_search.fit(X_train_transformed, y_train)

    # Get the best model
    best_log_reg = grid_search.best_estimator_

    # Evaluate the tuned model on validation set
    y_pred_best = best_log_reg.predict(X_valid_transformed)

    # Print the classification report
    print("Tuned Logistic Regression Classification Report:")
    print(classification_report(y_valid, y_pred_best))

    # Calculate precision, recall, and F1-score
    precision = precision_score(y_valid, y_pred_best)
    recall = recall_score(y_valid, y_pred_best)
    f1 = f1_score(y_valid, y_pred_best)

    # Print calculated metrics
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

    # Create a valid input example with the correct number of features
    input_example = np.random.rand(1, 86)  # Example with 86 random features

    # Log the tuned model in MLflow with the valid input example
    mlflow.sklearn.log_model(best_log_reg, "Tuned_Logistic_Regression_Model", input_example=input_example)

    # Log the parameters and metrics
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("test_score_tuned", accuracy_score(y_valid, y_pred_best))
    mlflow.log_metric("train_score_tuned", accuracy_score(y_train, best_log_reg.predict(X_train_transformed)))

    # Log additional metrics
    mlflow.log_metric("precision_tuned", precision)
    mlflow.log_metric("recall_tuned", recall)
    mlflow.log_metric("f1_score_tuned", f1)

    # Get MLflow run details
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 6: Save the model locally
    model_path = '/tmp/tuned_logistic_regression_model.pkl'
    joblib.dump(best_log_reg, model_path)

    # Step 7: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="Tuned_Logistic_Regression_Model",
        version=1,  # Version of the model
        metrics={"train_score": accuracy_score(y_train, best_log_reg.predict(X_train_transformed)), 
                  "validation_score": accuracy_score(y_valid, y_pred_best)},
        description="Tuned Logistic Regression Model for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("Tuned model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
Tuned Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.84      0.82       813
           1       0.66      0.60      0.63       413

    accuracy                           0.76      1226
   macro avg       0.73      0.72      0.72      1226
weighted avg       0.76      0.76      0.76      1226

Precision: 0.66
Recall: 0.60
F1 Score: 0.63
2024-10-19 19:37:55,294 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



MLflow Run ID: c9b52066dc36478599bc2319d4bed59d
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/c9b52066dc36478599bc2319d4bed59d


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1551 elapsed<00:00 remaining<?

2024/10/19 19:38:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run delightful-vole-998 at: http://127.0.0.1:5000/#/experiments/0/runs/c9b52066dc36478599bc2319d4bed59d.
2024/10/19 19:38:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/Tuned_Logistic_Regression_Model/1
Tuned model saved and registered in Hopsworks successfully!


B. Random Forest

Randome Forest with Hopsworks Model registry

In [56]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Step 2: Start MLflow run
with mlflow.start_run() as run:
    # Initialize and train the Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_transformed, y_train)

    # Make predictions on validation set
    y_pred_rf = rf_model.predict(X_valid_transformed)

    # Evaluate the model
    print("Random Forest Classification Report:")
    print(classification_report(y_valid, y_pred_rf))

    # Calculate and print accuracy scores
    train_predictions = rf_model.predict(X_train_transformed)
    train_score = accuracy_score(y_train, train_predictions)
    test_score = accuracy_score(y_valid, y_pred_rf)

    # Additional metrics
    precision = precision_score(y_valid, y_pred_rf, average='weighted')
    recall = recall_score(y_valid, y_pred_rf, average='weighted')
    f1 = f1_score(y_valid, y_pred_rf, average='weighted')

    print(f"Train Score (Accuracy): {train_score:.4f}")  # Train accuracy
    print(f"Validation/Test Score (Accuracy): {test_score:.4f}")  # Test accuracy

    # Log model and parameters in MLflow
    input_example = X_valid_transformed[0:1]  # Use one row from your validation set as an example

    # Log the model with an input example
    mlflow.sklearn.log_model(rf_model, "Random_Forest_Model", input_example=input_example)
    mlflow.log_params({
        "model": "Random Forest",
        "train_score": train_score,
        "test_score": test_score
    })
    mlflow.log_metrics({
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "train_accuracy": train_score,
        "test_accuracy": test_score
    })

    # Print overall accuracy of the model (validation/test)
    print(f"Overall Model Accuracy (Validation/Test): {test_score:.4f}")

    # Get the run ID for this run
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    # Display MLflow link to the run
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 6: Save the model locally
    model_path = '/tmp/random_forest_model.pkl'
    joblib.dump(rf_model, model_path)

    # Step 7: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="Random_Forest_Model",
        version=1,  # Version of the model
        metrics={"train_score": train_score, "validation_score": test_score},
        description="Random Forest Model for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("Random Forest model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       813
           1       0.78      0.58      0.66       413

    accuracy                           0.80      1226
   macro avg       0.79      0.75      0.76      1226
weighted avg       0.80      0.80      0.79      1226

Train Score (Accuracy): 0.9999
Validation/Test Score (Accuracy): 0.8010
2024-10-19 20:14:55,468 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



Overall Model Accuracy (Validation/Test): 0.8010
MLflow Run ID: 272c47e6b5e2493fb408b126a185a86e
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/272c47e6b5e2493fb408b126a185a86e


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/24537113 elapsed<00:00 remaining<?

2024/10/19 20:15:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run worried-skink-520 at: http://127.0.0.1:5000/#/experiments/0/runs/272c47e6b5e2493fb408b126a185a86e.
2024/10/19 20:15:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/Random_Forest_Model/1
Random Forest model saved and registered in Hopsworks successfully!


Random Forest Hyperparameter Tuning with Hopsworks Model Registry

In [58]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']  # Updated max_features values
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='accuracy', 
                           error_score='raise')

# Step 2: Start MLflow run for the grid search
with mlflow.start_run() as run:
    # Fit the grid search to the training data
    grid_search.fit(X_train_transformed, y_train)

    # Get the best model and hyperparameters
    best_rf_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Log the best hyperparameters
    mlflow.log_params(best_params)

    # Make predictions on the validation set
    y_pred_rf = best_rf_model.predict(X_valid_transformed)

    # Evaluate the model
    print("Random Forest Classification Report:")
    print(classification_report(y_valid, y_pred_rf))

    # Calculate and log accuracy scores
    train_score = accuracy_score(y_train, best_rf_model.predict(X_train_transformed))
    test_score = accuracy_score(y_valid, y_pred_rf)

    print(f"Train Score (Accuracy): {train_score:.4f}")  # Train accuracy
    print(f"Validation/Test Score (Accuracy): {test_score:.4f}")  # Test accuracy

    # Calculate additional metrics
    precision = precision_score(y_valid, y_pred_rf, average='weighted')
    recall = recall_score(y_valid, y_pred_rf, average='weighted')
    f1 = f1_score(y_valid, y_pred_rf, average='weighted')

    # Log the additional metrics
    mlflow.log_metrics({
        "train_accuracy": train_score,
        "test_accuracy": test_score,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

    # Log model with the best parameters
    input_example = X_valid_transformed[0:1]  # Use one row from your validation set as an example
    mlflow.sklearn.log_model(best_rf_model, "Tuned_Random_Forest_Model", input_example=input_example)

    # Get the run ID for this run
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    # Display MLflow link to the run
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 3: Save the model locally
    model_path = '/tmp/random_forest_model.pkl'
    joblib.dump(best_rf_model, model_path)

    # Step 4: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="Tuned_Random_Forest_Model",
        version=1,  # Version of the model
        metrics={"train_score": train_score, "validation_score": test_score},
        description="Random Forest Model with Hyperparameter Tuning for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("Random Forest model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       813
           1       0.78      0.58      0.66       413

    accuracy                           0.80      1226
   macro avg       0.80      0.75      0.76      1226
weighted avg       0.80      0.80      0.79      1226

Train Score (Accuracy): 0.9999
Validation/Test Score (Accuracy): 0.8034
2024-10-19 20:20:30,603 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



MLflow Run ID: 5f52e819cdd948808bd980601c1239c0
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/5f52e819cdd948808bd980601c1239c0


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/49174073 elapsed<00:00 remaining<?

2024/10/19 20:21:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-dove-971 at: http://127.0.0.1:5000/#/experiments/0/runs/5f52e819cdd948808bd980601c1239c0.
2024/10/19 20:21:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/Tuned_Random_Forest_Model/1
Random Forest model saved and registered in Hopsworks successfully!


C. XGBoost

with hopswork Modle registry

In [59]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Step 2: Start MLflow run
with mlflow.start_run() as run:
    # Initialize and train the XGBoost model
    xgb_model = XGBClassifier(eval_metric='logloss')  # Removed use_label_encoder
    xgb_model.fit(X_train_transformed, y_train)

    # Make predictions on the validation set
    y_pred_xgb = xgb_model.predict(X_valid_transformed)

    # Evaluate the model
    print("XGBoost Classification Report:")
    print(classification_report(y_valid, y_pred_xgb))
    
    # Calculate accuracy and other metrics
    accuracy = accuracy_score(y_valid, y_pred_xgb)
    precision = precision_score(y_valid, y_pred_xgb, average='weighted')
    recall = recall_score(y_valid, y_pred_xgb, average='weighted')
    f1 = f1_score(y_valid, y_pred_xgb, average='weighted')

    print(f"Validation/Test Accuracy: {accuracy:.4f}")
    
    # Calculate and log train score
    train_predictions = xgb_model.predict(X_train_transformed)
    train_accuracy = accuracy_score(y_train, train_predictions)
    print(f"Train Accuracy: {train_accuracy:.4f}")

    # Log model, parameters, and metrics in MLflow
    input_example = X_valid_transformed[0:1]  # Use one row from your validation set as an example
    mlflow.sklearn.log_model(xgb_model, "XGBoost_Model", input_example=input_example)  # Add input example

    mlflow.log_params({
        "model": "XGBoost",
        "eval_metric": 'logloss',
    })
    
    # Log the metrics
    mlflow.log_metrics({
        "train_accuracy": train_accuracy,
        "test_accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    })

    # Get the run ID for this run
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    # Display MLflow link to the run
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 3: Save the model locally
    model_path = '/tmp/xgboost_model.pkl'
    joblib.dump(xgb_model, model_path)

    # Step 4: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="XGBoost_Model",
        version=1,  # Version of the model
        metrics={"train_score": train_accuracy, "validation_score": accuracy},
        description="XGBoost Model for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("XGBoost model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       813
           1       0.73      0.60      0.65       413

    accuracy                           0.79      1226
   macro avg       0.77      0.74      0.75      1226
weighted avg       0.78      0.79      0.78      1226

Validation/Test Accuracy: 0.7879
Train Accuracy: 0.9540
2024-10-19 20:22:54,933 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



MLflow Run ID: 6bb53ca10fb248ad8b2a40e1c59fb7d3
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/6bb53ca10fb248ad8b2a40e1c59fb7d3


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/322474 elapsed<00:00 remaining<?

2024/10/19 20:23:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run delicate-kit-177 at: http://127.0.0.1:5000/#/experiments/0/runs/6bb53ca10fb248ad8b2a40e1c59fb7d3.
2024/10/19 20:23:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/XGBoost_Model/1
XGBoost model saved and registered in Hopsworks successfully!


hyperparameter tuning - XG Boost

tuned_XGBoost with model registry and mlflow

In [60]:
# Import necessary libraries
import hsml
import mlflow
import mlflow.sklearn
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import joblib

# Step 1: Establish a connection to Hopsworks using host, project name, and API key
connection = hsml.connection(
    host="c.app.hopsworks.ai",  # Your Hopsworks instance URL
    project="ML_truckdelay",  # Correct project name from Hopsworks
    api_key_value="8ViXMBkcrCAUFm4Y.PEDm3J6NPoMDtf2L7MpnYsCdElsDMCPRr1RI3vbPtLnjogWE1f6hdKOoxpQbxrM0"
)

# Access the model registry
mr = connection.get_model_registry()

# Set MLflow tracking URI (update if MLflow is hosted remotely)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Step 2: Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [None, 10, 20],  # Maximum depth of trees
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'subsample': [0.8, 1.0],  # Subsample ratio
    'colsample_bytree': [0.8, 1.0]  # Subsample ratio of columns
}

# Initialize the XGBoost model
xgb_model = XGBClassifier(eval_metric='logloss')

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='accuracy', 
                           error_score='raise')

# Start MLflow run
with mlflow.start_run() as run:
    # Fit the grid search to the training data
    grid_search.fit(X_train_transformed, y_train)

    # Get the best model and hyperparameters
    best_xgb_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Log the best hyperparameters
    mlflow.log_params(best_params)

    # Make predictions on the validation set
    y_pred_xgb = best_xgb_model.predict(X_valid_transformed)

    # Evaluate the model
    print("XGBoost Classification Report:")
    print(classification_report(y_valid, y_pred_xgb))
    
    # Calculate accuracy and other metrics
    accuracy = accuracy_score(y_valid, y_pred_xgb)
    precision = precision_score(y_valid, y_pred_xgb, average='weighted')
    recall = recall_score(y_valid, y_pred_xgb, average='weighted')
    f1 = f1_score(y_valid, y_pred_xgb, average='weighted')

    print(f"Validation/Test Accuracy: {accuracy:.4f}")
    
    # Calculate and log train score
    train_predictions = best_xgb_model.predict(X_train_transformed)
    train_accuracy = accuracy_score(y_train, train_predictions)
    print(f"Train Accuracy: {train_accuracy:.4f}")

    # Log model, parameters, and metrics in MLflow
    input_example = X_valid_transformed[0:1]  # Use one row from your validation set as an example
    mlflow.sklearn.log_model(best_xgb_model, "XGBoost_Tuned_Model", input_example=input_example)  # Add input example

    # Log the metrics
    mlflow.log_metrics({
        "train_accuracy": train_accuracy,
        "test_accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    })

    # Get the run ID for this run
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    # Display MLflow link to the run
    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")

    # Step 3: Save the model locally
    model_path = '/tmp/tuned_xgboost_model.pkl'
    joblib.dump(best_xgb_model, model_path)

    # Step 4: Register the model in Hopsworks Model Registry
    model = mr.sklearn.create_model(
        name="Tuned_XGBoost_Model",
        version=1,  # Version of the model
        metrics={"train_score": train_accuracy, "validation_score": accuracy},
        description="Tuned XGBoost Model for Truck Delay Classification"
    )

    # Save the model to the registry
    model.save(model_path)

    print("Tuned XGBoost model saved and registered in Hopsworks successfully!")


Connected. Call `.close()` to terminate connection gracefully.
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       813
           1       0.80      0.59      0.68       413

    accuracy                           0.81      1226
   macro avg       0.81      0.76      0.77      1226
weighted avg       0.81      0.81      0.80      1226

Validation/Test Accuracy: 0.8116
Train Accuracy: 0.9999
2024-10-19 20:26:49,871 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



MLflow Run ID: 1a27a7639fe7415bbddaae9e47b794f5
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/1a27a7639fe7415bbddaae9e47b794f5


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/2691337 elapsed<00:00 remaining<?

2024/10/19 20:27:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run judicious-kit-471 at: http://127.0.0.1:5000/#/experiments/0/runs/1a27a7639fe7415bbddaae9e47b794f5.
2024/10/19 20:27:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Model created, explore it at https://c.app.hopsworks.ai:443/p/1104021/models/Tuned_XGBoost_Model/1
Tuned XGBoost model saved and registered in Hopsworks successfully!


stacking ensemble method

Model Initialization and Training

In [91]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import mlflow.sklearn

# Assuming X_train_transformed, y_train, X_valid_transformed, y_valid are already defined

# Initialize base models
logistic_regression = LogisticRegression(max_iter=1000)
random_forest = RandomForestClassifier(random_state=42)
xgboost = XGBClassifier(eval_metric='logloss')

# Train base models and obtain predictions
log_reg_preds = logistic_regression.fit(X_train_transformed, y_train).predict(X_train_transformed)
rf_preds = random_forest.fit(X_train_transformed, y_train).predict(X_train_transformed)
xgb_preds = xgboost.fit(X_train_transformed, y_train).predict(X_train_transformed)

# Combine base model predictions into a single DataFrame for training the meta-model
base_model_preds = np.column_stack((log_reg_preds, rf_preds, xgb_preds))

# For validation predictions
log_reg_preds_valid = logistic_regression.predict(X_valid_transformed)
rf_preds_valid = random_forest.predict(X_valid_transformed)
xgb_preds_valid = xgboost.predict(X_valid_transformed)

# Combine validation predictions into a single DataFrame for the meta-model
base_model_preds_valid = np.column_stack((log_reg_preds_valid, rf_preds_valid, xgb_preds_valid))


Meta-Model Initialization, Training, and Logging - using logistic regression

In [90]:
# Initialize and train the meta-model (e.g., Logistic Regression)
meta_model = LogisticRegression()

# Fit the meta-model on the base model predictions
meta_model.fit(base_model_preds, y_train)

# Make predictions on the validation set using the meta-model
y_pred_meta = meta_model.predict(base_model_preds_valid)

# Evaluate the meta-model
print("Meta-Model (Logistic Regression) Classification Report:")
print(classification_report(y_valid, y_pred_meta))
print(f"Meta-Model Accuracy: {accuracy_score(y_valid, y_pred_meta):.4f}")

# Calculate and log the training accuracy
train_preds_meta = meta_model.predict(base_model_preds)
train_accuracy = accuracy_score(y_train, train_preds_meta)

# Start a nested run
with mlflow.start_run(nested=True) as run:
    # Log the meta-model
    mlflow.sklearn.log_model(meta_model, "Meta_Model_Classifier")
    mlflow.log_params({"model": "Logistic Regression Meta-Model"})
    mlflow.log_metric("meta_model_accuracy", accuracy_score(y_valid, y_pred_meta))
    mlflow.log_metric("meta_model_train_accuracy", train_accuracy)  # Log train accuracy

    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")


Meta-Model (Logistic Regression) Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       813
           1       0.78      0.58      0.66       413

    accuracy                           0.80      1226
   macro avg       0.79      0.75      0.76      1226
weighted avg       0.80      0.80      0.79      1226

Meta-Model Accuracy: 0.8010
2024-10-18 15:01:10,886 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



2024/10/18 15:01:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 15:01:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run serious-fish-850 at: http://127.0.0.1:5000/#/experiments/0/runs/fd54ac89635946ba9412417860e3aad2.
2024/10/18 15:01:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


MLflow Run ID: fd54ac89635946ba9412417860e3aad2
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/fd54ac89635946ba9412417860e3aad2


Using Random Forest as Meta-Model

In [92]:
# Initialize and train the meta-model (Random Forest)
meta_model = RandomForestClassifier(random_state=42)

# Fit the meta-model on the base model predictions
meta_model.fit(base_model_preds, y_train)

# Make predictions on the validation set using the meta-model
y_pred_meta = meta_model.predict(base_model_preds_valid)

# Evaluate the meta-model
print("Meta-Model (Random Forest) Classification Report:")
print(classification_report(y_valid, y_pred_meta))
print(f"Meta-Model Accuracy: {accuracy_score(y_valid, y_pred_meta):.4f}")

# Calculate and log the training accuracy
train_preds_meta = meta_model.predict(base_model_preds)
train_accuracy = accuracy_score(y_train, train_preds_meta)

# Start a nested run
with mlflow.start_run(nested=True) as run:
    # Log the meta-model
    mlflow.sklearn.log_model(meta_model, "Meta_Model_Classifier_RF")
    mlflow.log_params({"model": "Random Forest Meta-Model"})
    mlflow.log_metric("meta_model_accuracy", accuracy_score(y_valid, y_pred_meta))
    mlflow.log_metric("meta_model_train_accuracy", train_accuracy)  # Log train accuracy

    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")


Meta-Model (Random Forest) Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       813
           1       0.78      0.58      0.66       413

    accuracy                           0.80      1226
   macro avg       0.79      0.75      0.76      1226
weighted avg       0.80      0.80      0.79      1226

Meta-Model Accuracy: 0.8010
2024-10-18 15:02:44,958 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



2024/10/18 15:02:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 15:02:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-fly-476 at: http://127.0.0.1:5000/#/experiments/0/runs/b8e0bcca827a453785502517b2bce9ef.
2024/10/18 15:02:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


MLflow Run ID: b8e0bcca827a453785502517b2bce9ef
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/b8e0bcca827a453785502517b2bce9ef


Using XGBoost as Meta-Model

In [93]:
# Initialize and train the meta-model (XGBoost)
meta_model = XGBClassifier(eval_metric='logloss', random_state=42)

# Fit the meta-model on the base model predictions
meta_model.fit(base_model_preds, y_train)

# Make predictions on the validation set using the meta-model
y_pred_meta = meta_model.predict(base_model_preds_valid)

# Evaluate the meta-model
print("Meta-Model (XGBoost) Classification Report:")
print(classification_report(y_valid, y_pred_meta))
print(f"Meta-Model Accuracy: {accuracy_score(y_valid, y_pred_meta):.4f}")

# Calculate and log the training accuracy
train_preds_meta = meta_model.predict(base_model_preds)
train_accuracy = accuracy_score(y_train, train_preds_meta)

# Start a nested run
with mlflow.start_run(nested=True) as run:
    # Log the meta-model
    mlflow.sklearn.log_model(meta_model, "Meta_Model_Classifier_XGB")
    mlflow.log_params({"model": "XGBoost Meta-Model"})
    mlflow.log_metric("meta_model_accuracy", accuracy_score(y_valid, y_pred_meta))
    mlflow.log_metric("meta_model_train_accuracy", train_accuracy)  # Log train accuracy

    run_id = run.info.run_id
    experiment_id = run.info.experiment_id

    print(f"MLflow Run ID: {run_id}")
    print(f"MLflow Link: http://127.0.0.1:5000/#/experiments/{experiment_id}/runs/{run_id}")


Meta-Model (XGBoost) Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       813
           1       0.78      0.58      0.66       413

    accuracy                           0.80      1226
   macro avg       0.79      0.75      0.76      1226
weighted avg       0.80      0.80      0.79      1226

Meta-Model Accuracy: 0.8010
2024-10-18 15:03:21,937 WARNING: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



2024/10/18 15:03:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/18 15:03:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run delightful-gnat-382 at: http://127.0.0.1:5000/#/experiments/0/runs/94633cf22c6f426b90794159688742d8.
2024/10/18 15:03:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


MLflow Run ID: 94633cf22c6f426b90794159688742d8
MLflow Link: http://127.0.0.1:5000/#/experiments/0/runs/94633cf22c6f426b90794159688742d8
